[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/%E7%AC%AC%E5%9B%9B%E7%AB%A0_tf%E5%AE%9E%E7%8E%B0%E8%87%AA%E7%BC%96%E7%A0%81%E5%99%A8%E5%8F%8A%E5%A4%9A%E5%B1%82%E6%84%9F%E7%9F%A5%E6%9C%BA.ipynb)

## 自编码器简介
**传统机器学习任务很大程度上依赖于好的特征工程**，比如对数值型、日期时间型、种类型等特征的提取。特征工程往往是非常耗时耗力的，在图像、语音和视频中提取到有效的特征就更难了，工程师必须在这些领域有非常深入的理解，并且使用专业算法提取这些数据的特征。**深度学习则可以解决人工难以提取有效特征的问题，它可以大大缓解机器学习特征对特征工程的依赖**。深度学习在早期一度被认为是一种无监督的特征学习，**模仿了人脑对特征逐层抽象提取的过程**。这其中有两点很重要：
1. 无监督学习，即我们不需要标注数据就可以对数据进行一定程度的学习，这种学习是对数据内容的组织形式的学习，提取的是频繁出现的特征
2. 逐层抽象，特征是需要不断抽象的，就像人总是从简单基础的概念开始学习，再到复杂的概念。

自编码器(AutoEncoder),顾名思义，即可以使用自身的高阶特征编码自己。编码器其实也是一种神经网络，它的输入和输出是一致的，它借助稀疏编码的思想。目标是使用系数的一些高阶特征重新组合来重构自己。因此，它的特点非常明显；第一，期望输入／输出一致；第二，希望使用高阶特征来重构自己，而不只是复制像素点。

[参考 Hinton教授的文章 reducing the dimensionality of data with neural networks](http://science.sciencemag.org/content/313/5786/504)

hinton还提出了机遇深度信念网络(deep belief networks,dbn,由多层rbm堆叠而成)可使用无监督的逐层训练的贪心算法，为训练很深的网络提供了一个可行方案：我们可能很难直接训练极深的网络，但是可以用无监督的逐层训练提取特征，将网络的圈中初始化到一个比较好的位置，辅助后面的监督训练。无监督的逐层训练，其思想和自编码器非常相似。

In [0]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
def xavier_init(fan_in, fan_out, constant=1):
  low = -constant * np.sqrt(6.0/(fan_in+fan_out))
  high = constant*np.sqrt(6.0/(fan_in+fan_out))
  return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

In [0]:
class AdditiveGaussianNoiseAutoencoder(object):
  def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus,optimizer=tf.train.AdamOptimizer(), scale=0.1):
    self.n_input = n_input #输入变量数
    self.n_hidden = n_hidden #隐藏层节点数
    self.transfer = transfer_function #隐藏层激活函数
    self.scale = tf.placeholder(tf.float32)
    self.training_scale = scale
    network_weights = self._initialize_weights() #参数初始化函数
    self.weights=network_weights
    
    self.x = tf.placeholder(tf.float32, [None,self.n_input]) #n_input维的输入x
    #先将输入x加入噪声，即self.x+scale*tf.random_normal((n_input,))一个能提取特征的隐藏层
    self.hidden = self.transfer(tf.add(tf.matmul(self.x+scale*tf.random_normal((n_input,)), self.weights['w1']), self.weights['b1']))
    #经过隐藏层后，我们需要在输出层进行数据复原、重建操作(即建立reconstruction层)，即直接将隐藏层的输出self.hidden乘上输出层的权重w2，再加上输出层的偏置b2即可。
    self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
    self.cost = 0.5*tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2.0)) #定义自编码器的损失函数，这里直接使用平方误差作为cost
    self.optimizer = optimizer.minimize(self.cost) #优化器
    
    #创建session，并初始化自编码器的全部模型参数
    init = tf.global_variables_initializer()
    self.sess = tf.Session()
    self.sess.run(init)
    
  def _initialize_weights(self):
    all_weights = dict()
    all_weights['w1'] = tf.Variable(xavier_init(self.n_input,self.n_hidden))
    all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden],dtype=tf.float32))
    all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input], dtype=tf.float32))
    all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],dtype=tf.float32))
    
    return all_weights
  
  def partial_fit(self, X):
    #功能:用一个batch数据进行训练并返回当前损失cost
    cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict={self.x:X, self.scale:self.training_scale})
    return cost
  
  def calc_total_cost(self,X):
    #功能：在自编码器训练完毕后，在测试集上对模型性能进行测评时会用到的，它不会像partial_fit那样触发训练操作
    return self.sess.run(self.cost, feed_dict={self.x:X, self.scale:self.training_scale})
  
  def transform(self, X):
    #功能:返回自编码器隐藏层的输出结果，目的是提供一个接口来获取抽象后的特征，自编码器的隐藏层的最主要功能就是学习出数据中的高阶特征
    return self.sess.run(self.hidden, feed_dict={self.x:X, self.scale:self.training_scale})
  
  def generate(self, hidden=None):
    #功能：将隐藏层的输出结构作为输入，通过之后的重建层将提取到的高阶特征复原为原始数据。
    if hidden is None:
      hidden = np.random.normal(size=self.weights['b1'])
      
    return self.sess.run(self.reconstruction, feed_dict={self.hidden:hidden})
  
  def reconstruct(self, X):
    #功能:它整体运行一遍复原过程，包括提取高阶特征和通过高阶特征复原数据，即包括transform和generate两块
    return self.sess.run(self.reconstruction, feed_dict={self.x:X, self.scale:self.training_scale})
  
  def getWeights(self):
    #功能:获取隐藏层的权重w1
    return self.sess.run(self.weights['w1'])
  
  def getBiases(self):
    #功能:获取隐藏层的偏置系数b1
    return self.sess.run(self.weights['b1'])

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [0]:
#数据集进行标准化缩放
def standard_scale(X_train, X_test):
  preprocessor = prep.StandardScaler().fit(X_train)
  X_train = preprocessor.transform(X_train)
  X_test = preprocessor.transform(X_test)
  return X_train, X_test

In [0]:
#放回抽样，获取随机block数据的函数
def get_random_block_from_data(data, batch_size):
  start_index = np.random.randint(0, len(data)-batch_size)
  return data[start_index:(start_index+batch_size)]

In [0]:
X_train,X_test = standard_scale(mnist.train.images, mnist.test.images)

In [0]:
n_samples = int(mnist.train.num_examples)
training_epochs=20 #最大训练的轮数
batch_size = 128
display_step = 1

In [0]:
#创建一个AGN自编码器的实例，定义模型输入节点数n_input为784，自编码器的隐含层节点数n_hidden为200，隐藏层的激活函数transfer_function为softplus
#优化器optimizer为Adam且学习速率为0.001，同时将噪声的系数scale设为0.01

autoencoder = AdditiveGaussianNoiseAutoencoder(n_input=784,n_hidden=200,transfer_function=tf.nn.softplus,
                                              optimizer=tf.train.AdamOptimizer(learning_rate=0.001),scale=0.01)

In [13]:
#开始训练，在每一轮epoch循环开始时，将平均损失设为0，并计算总共需要的batch数
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch=int(n_samples/batch_size)
  for i in range(total_batch):
    batch_xs = get_random_block_from_data(X_train, batch_size)
    
    cost = autoencoder.partial_fit(batch_xs)
    avg_cost += cost/n_samples*batch_size
    
  if epoch % display_step ==0:
    print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

Epoch: 0001 cost= 12228.043276136
Epoch: 0002 cost= 10514.235121591
Epoch: 0003 cost= 10024.321870455
Epoch: 0004 cost= 10651.358263068
Epoch: 0005 cost= 10086.104056250
Epoch: 0006 cost= 9202.270382955
Epoch: 0007 cost= 8035.978659659
Epoch: 0008 cost= 8924.483802273
Epoch: 0009 cost= 8939.779858523
Epoch: 0010 cost= 8700.661833523
Epoch: 0011 cost= 9204.292671591
Epoch: 0012 cost= 8507.349712500
Epoch: 0013 cost= 8126.556160795
Epoch: 0014 cost= 8889.838689773
Epoch: 0015 cost= 8343.000498864
Epoch: 0016 cost= 8417.482429545
Epoch: 0017 cost= 7733.393575568
Epoch: 0018 cost= 7790.606960227
Epoch: 0019 cost= 7938.459438636
Epoch: 0020 cost= 8883.212060795


In [14]:
print("Total cost:"+str(autoencoder.calc_total_cost(X_test)))

Total cost:707946.44


## 多层感知机

In [0]:
sess = tf.InteractiveSession()

In [0]:
in_units = 784 #输入节点数
h1_units = 300 #隐藏层的输出节点数
#隐藏层的权重和偏置
#将权重初始化为截断的正态分布，其标准差为0.1(因为使用的激活函数是ReLU，所以需要正态分布给参数增加一点噪声来打破完全对称并且避免0梯度)
W1 = tf.Variable(tf.truncated_normal([in_units,h1_units],stddev=0.1)) 
b1 = tf.Variable(tf.zeros([h1_units]))
#输出层的权重和偏置
W2 = tf.Variable(tf.zeros([h1_units,10]))
b2 = tf.Variable(tf.zeros([10]))

In [0]:
x = tf.placeholder(tf.float32, [None,in_units])
#dropout的比率keep_prob(即保留节点的概率)是不一样的，通常在训练时小于1，而预测时则等于1
keep_prob = tf.placeholder(tf.float32)

In [0]:
#定义模型结构。首先需要一个隐藏层，为hidden1，接着，调用tf.nn.dropout实现dropout功能，即随机将一部分节点置为0。
#这里的keep_prob参数即为保留数据而不置为0的比例，在训练时应该是小于1的，用于制造随机性，防止过拟合，最后是输出层

hidden1 = tf.nn.relu(tf.matmul(x,W1)+b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, W2)+b2)

In [0]:
#定义损失函数和选择优化器来优化loss，这里的损失函数继续使用交叉熵，而优化器使用Adagrad，学习速率设为0.3
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y),reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

In [0]:
tf.global_variables_initializer().run()
for i in range(3000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run({x:batch_xs, y_:batch_ys,keep_prob:0.75})

In [22]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

0.9784
